## **Notebook para implementar la regresión con minimización del MAPE.**

Carga de las librerías que se usarán en el análisis.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Cargar los datos

In [0]:
pathfile = '...'
data = pd.read_csv(pathfile)

Declaración de funciones